In [ ]:
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib  import Path
import os

In [ ]:
import zipfile

# Step 2: Extract ZIP from Drive
import zipfile
zip_path = "/content/drive/MyDrive/Colab Notebooks/archive (6).zip"  # change if needed
extract_dir = "/content/data"

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)


In [ ]:
train_ds_path = '/content/data/chest_xray/train'
val_ds_path = '/content/data/chest_xray/val'

In [ ]:
train_ds = tf.keras.utils.image_dataset_from_directory(
    train_ds_path,
    seed=123,
    image_size=(224, 224),
    batch_size=32,
    shuffle=True,
    label_mode='binary'
)


Found 5216 files belonging to 2 classes.


In [ ]:
val_ds=tf.keras.preprocessing.image_dataset_from_directory(
    val_ds_path,
    seed=123,
    image_size=(224, 224),
    batch_size=32,
    shuffle=True,
    label_mode='binary'
)

Found 16 files belonging to 2 classes.


In [ ]:
# Data augmentation for training only
data_augmentation = tf.keras.Sequential([
    tf.keras.layers.RandomFlip("horizontal"),
    tf.keras.layers.RandomRotation(0.2),
    tf.keras.layers.RandomZoom(0.2),
])

# Load pre-trained EfficientNetB0 model
base_model = tf.keras.applications.EfficientNetB0(
    include_top=False,
    weights="imagenet",
    input_shape=(224, 224, 3)
)
base_model.trainable = False

# Define the final model
model = tf.keras.Sequential([
    tf.keras.layers.InputLayer(input_shape=(224, 224, 3)),
    data_augmentation,
    base_model,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()


16705208/16705208 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/input_layer.py:27: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ sequential (Sequential)         │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ efficientnetb0 (Functional)     │ (None, 7, 7, 1280)     │     4,049,571 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 1280)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1)              │         1,281 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,050,852 (15.45 MB)

 Trainable params: 1,281 (5.00 KB)

 Non-trainable params: 4,049,571 (15.45 MB)

In [ ]:
model.fit(train_ds, epochs=10, validation_data=val_ds)

Epoch 1/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 504s 3s/step - accuracy: 0.7616 - loss: 0.4728 - val_accuracy: 0.6875 - val_loss: 0.5275
Epoch 2/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 491s 3s/step - accuracy: 0.8862 - loss: 0.2689 - val_accuracy: 0.6875 - val_loss: 0.5512
Epoch 3/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 490s 3s/step - accuracy: 0.8936 - loss: 0.2364 - val_accuracy: 0.8750 - val_loss: 0.3880
Epoch 4/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 498s 3s/step - accuracy: 0.9063 - loss: 0.2162 - val_accuracy: 0.6875 - val_loss: 0.5276
Epoch 5/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 499s 3s/step - accuracy: 0.9176 - loss: 0.2005 - val_accuracy: 0.8750 - val_loss: 0.3799
Epoch 6/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 492s 3s/step - accuracy: 0.9240 - loss: 0.1869 - val_accuracy: 0.8750 - val_loss: 0.3992
Epoch 7/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 488s 3s/step - accuracy: 0.9175 - loss: 0.1946 - val_accuracy: 0.8750 - val_loss: 0.3609
Epoch 8/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 495s 3s/step - accuracy: 0.9197 - loss: 0.1893 - val_accu